In [1]:
!pip install cmake


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install face_recognition

In [ ]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
def play_siren_sound():
    def _sound_worker():
        sys_plat = platform.system()
        try:
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- classify_action ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        L_ANKLE = mp_holistic.PoseLandmark.LEFT_ANKLE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]
        l_ankle = landmarks[L_ANKLE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        # 1. Wave Detection
        if l_wrist.visibility > 0.5 and lw_y < nose_y:
            return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y:
            return "Wave Right"
            
        # 2. Sit/Stand Detection
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: # Thigh is horizontal
                return "Sit"
            else:
                return "Standing"

        return "Standing" 

    except Exception as e:
        return "Unknown"

# --- Helper: IoU for Overlap Check ---
def calculate_iou(boxA, boxB):
    # box = (x, y, w, h) -> convert to (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-5)
    return iou

# --- Tkinter Application Class ---
class PoseApp:
    def _init_(self, window_title="Pose Guard (Multi-Target)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.RESIZE_SCALE = 1.0 
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            logger.info("MediaPipe Holistic Loaded.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Holistic Model: {e}")
            self.root.destroy()
            return

        self.frame_timestamp_ms = 0 

        # --- Layout ---
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        # 1. Red Zone
        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        # Bottom Container
        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        # 2. Yellow Zone
        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        # 3. Green Zone
        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Widgets
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')

        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets to Track (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.mainloop()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 4:
                    display_name = " ".join(parts[1:-2])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except Exception as e:
                logger.error(f"Error parsing {f}: {e}")

        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children():
            widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections:
            tk.Label(self.preview_display, text="No Selection", bg="black", fg="white").pack(expand=True)
            return
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    target_h = 130 if len(display_idx) > 1 else 260
                    target_w = 180 if len(display_idx) > 1 else 360
                    h, w = img.shape[:2]
                    scale = min(target_w/w, target_h/h)
                    new_w, new_h = int(w*scale), int(h*scale)
                    img = cv2.resize(img, (new_w, new_h))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except Exception: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections:
            messagebox.showwarning("Selection", "No targets selected.")
            return
        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    target_image_file = face_recognition.load_image_file(filename)
                    encodings = face_recognition.face_encodings(target_image_file)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 # NEW: Counter for missing body
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        if count > 0:
            logger.info(f"Tracking initialized for {count} targets.")
            messagebox.showinfo("Tracking Updated", f"Now scanning for {count} selected targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging:
                self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Enter seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except Exception: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, bgr_frame)
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    def snap_photo(self):
        if self.unprocessed_frame is None: return
        rgb_frame = cv2.cvtColor(self.unprocessed_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        if len(face_locations) == 1:
             name = simpledialog.askstring("Name", "Enter Name:")
             if name:
                 safe_name = name.strip().replace(" ", "_")
                 cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
                 self.load_targets()
                 self.exit_capture_mode()
        else:
            messagebox.showwarning("Error", "Ensure exactly one face is visible.")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
        self.unprocessed_frame = frame.copy()
        if self.is_in_capture_mode:
            self.process_capture_frame(frame)
        else:
            self.process_tracking_frame_optimized(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # --- Full Fill Resize Logic ---
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame.shape[:2]
                # Maintain aspect ratio
                scale = min(lbl_w/w, lbl_h/h)
                new_w, new_h = int(w*scale), int(h*scale)
                frame_rgb = cv2.resize(frame_rgb, (new_w, new_h))
            
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
        self.root.after(10, self.update_video_feed)

    def process_capture_frame(self, frame):
        h, w = frame.shape[:2]
        cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        return frame

    # --- TRACKING LOGIC ---
    def process_tracking_frame_optimized(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL:
            self.re_detect_counter = 0
        
        rgb_full_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x + w, y + h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection (GREEDY BEST MATCH) - Fixes Target Switching
        untracked_targets = [name for name, s in self.targets_status.items() if not s["visible"]]
        
        if untracked_targets and self.re_detect_counter == 0:
            face_locations = face_recognition.face_locations(rgb_full_frame)
            if face_locations:
                face_encodings = face_recognition.face_encodings(rgb_full_frame, face_locations)
                possible_matches = []
                
                for i, unknown_encoding in enumerate(face_encodings):
                    for name in untracked_targets:
                        target_encoding = self.targets_status[name]["encoding"]
                        dist = face_recognition.face_distance([target_encoding], unknown_encoding)[0]
                        if dist < 0.55:
                            possible_matches.append((dist, i, name))
                
                possible_matches.sort(key=lambda x: x[0])
                assigned_faces = set()
                assigned_targets = set()
                
                for dist, face_idx, name in possible_matches:
                    if face_idx in assigned_faces or name in assigned_targets: continue
                    
                    assigned_faces.add(face_idx)
                    assigned_targets.add(name)
                    (top, right, bottom, left) = face_locations[face_idx]
                    
                    tracker = cv2.legacy.TrackerCSRT_create()
                    tracker.init(frame, (left, top, right-left, bottom-top))
                    self.targets_status[name]["tracker"] = tracker
                    self.targets_status[name]["face_box"] = (left, top, right, bottom)
                    self.targets_status[name]["visible"] = True
                    self.targets_status[name]["missing_pose_counter"] = 0

        # 3. Overlap Check (Fixes Merging Targets)
        active_names = [n for n, s in self.targets_status.items() if s["visible"]]
        for i in range(len(active_names)):
            for j in range(i + 1, len(active_names)):
                nameA = active_names[i]
                nameB = active_names[j]
                
                # Check Face Box IoU
                boxA = self.targets_status[nameA]["face_box"]
                boxB = self.targets_status[nameB]["face_box"]
                # Convert to x,y,w,h format for IoU check
                rectA = (boxA[0], boxA[1], boxA[2]-boxA[0], boxA[3]-boxA[1])
                rectB = (boxB[0], boxB[1], boxB[2]-boxB[0], boxB[3]-boxB[1])
                
                iou = calculate_iou(rectA, rectB)
                if iou > 0.5: # Significant overlap
                    # Force re-detection for both
                    self.targets_status[nameA]["tracker"] = None
                    self.targets_status[nameA]["visible"] = False
                    self.targets_status[nameB]["tracker"] = None
                    self.targets_status[nameB]["visible"] = False

        # 4. Processing & Drawing
        required_act = self.required_action_var.get()
        current_time = time.time()

        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                
                # --- CALCULATE BODY BOX (Torso-Centric) ---
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                # Ghost Box Check: Only draw if tracker is confident AND pose is found
                pose_found_in_box = False
                
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        results_crop = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        current_action = "Unknown"
                        if results_crop.pose_landmarks:
                            pose_found_in_box = True
                            status["missing_pose_counter"] = 0 # Reset
                            
                            draw_styled_landmarks(crop, results_crop)
                            raw_action = classify_action(results_crop.pose_landmarks.landmark, (by2-by1), (bx2-bx1))
                            
                            status["pose_buffer"].append(raw_action)
                            if len(status["pose_buffer"]) >= 8:
                                most_common = Counter(status["pose_buffer"]).most_common(1)[0][0]
                                current_action = most_common
                            else:
                                current_action = raw_action

                            if current_action == required_act:
                                if self.is_alert_mode:
                                    status["last_wave_time"] = current_time
                                    status["alert_triggered_state"] = False
                                if self.is_logging and status["last_logged_action"] != required_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, current_action, "SAFE (Reset)", "N/A"))
                                    status["last_logged_action"] = required_act
                            elif status["last_logged_action"] == required_act:
                                status["last_logged_action"] = None
                            
                            # Draw Box ONLY if pose found
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {current_action}", (bx1, by1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Ghost Box Removal Logic
                if not pose_found_in_box:
                    status["missing_pose_counter"] += 1
                    # If tracker says visible, but no pose for 5 frames -> Kill Tracker
                    if status["missing_pose_counter"] > 5:
                        status["tracker"] = None
                        status["visible"] = False

            # Alert Logic
            if self.is_alert_mode:
                time_diff = current_time - status["last_wave_time"]
                time_left = max(0, self.alert_interval - time_diff)
                y_offset = 50 + (list(self.targets_status.keys()).index(name) * 30)
                color = (0, 255, 0) if time_left > 3 else (0, 0, 255)
                
                # Only show status on screen if target is genuinely lost or safe
                status_txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({status_txt}): {time_left:.1f}s", (frame_w - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                if time_diff > self.alert_interval:
                    if (current_time - status["alert_cooldown"]) > 2.5:
                        play_siren_sound()
                        status["alert_cooldown"] = current_time
                        
                        img_path = "N/A"
                        if status["visible"]:
                            # Snapshot logic (Body box re-calc)
                            fx1, fy1, fx2, fy2 = status["face_box"]
                            face_w = fx2 - fx1
                            bx1 = max(0, int(fx1 + (face_w//2) - (face_w * 3)))
                            bx2 = min(frame_w, int(fx1 + (face_w//2) + (face_w * 3)))
                            by1 = max(0, int(fy1 - (face_w * 0.5)))
                            by2 = frame_h
                            if bx1 < bx2:
                                img_path = self.capture_alert_snapshot(frame[by1:by2, bx1:bx2], name)
                        else:
                            img_path = self.capture_alert_snapshot(frame, name)

                        if self.is_logging:
                            log_s = "ALERT CONTINUED" if status["alert_triggered_state"] else "ALERT TRIGGERED"
                            log_a = current_action if status["visible"] else "MISSING"
                            self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, log_a, log_s, img_path))
                            status["alert_triggered_state"] = True

        return frame 

# CORRECT
if __name__ == "__main__":
    
  app = PoseApp()

In [2]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
def play_siren_sound():
    def _sound_worker():
        sys_plat = platform.system()
        try:
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- classify_action ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        L_ANKLE = mp_holistic.PoseLandmark.LEFT_ANKLE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]
        l_ankle = landmarks[L_ANKLE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        # 1. Wave Detection
        if l_wrist.visibility > 0.5 and lw_y < nose_y:
            return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y:
            return "Wave Right"
            
        # 2. Sit/Stand Detection
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: # Thigh is horizontal
                return "Sit"
            else:
                return "Standing"

        return "Standing" 

    except Exception as e:
        return "Unknown"

# --- Helper: IoU for Overlap Check ---
def calculate_iou(boxA, boxB):
    # box = (x, y, w, h) -> convert to (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-5)
    return iou

# --- Tkinter Application Class ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Multi-Target)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.RESIZE_SCALE = 1.0 
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            logger.info("MediaPipe Holistic Loaded.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Holistic Model: {e}")
            self.root.destroy()
            return

        self.frame_timestamp_ms = 0 

        # --- Layout ---
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        # 1. Red Zone
        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        # Bottom Container
        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        # 2. Yellow Zone
        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        # 3. Green Zone
        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Widgets
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')

        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets to Track (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.mainloop()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 4:
                    display_name = " ".join(parts[1:-2])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except Exception as e:
                logger.error(f"Error parsing {f}: {e}")

        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children():
            widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections:
            tk.Label(self.preview_display, text="No Selection", bg="black", fg="white").pack(expand=True)
            return
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    target_h = 130 if len(display_idx) > 1 else 260
                    target_w = 180 if len(display_idx) > 1 else 360
                    h, w = img.shape[:2]
                    scale = min(target_w/w, target_h/h)
                    new_w, new_h = int(w*scale), int(h*scale)
                    img = cv2.resize(img, (new_w, new_h))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except Exception: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections:
            messagebox.showwarning("Selection", "No targets selected.")
            return
        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    target_image_file = face_recognition.load_image_file(filename)
                    encodings = face_recognition.face_encodings(target_image_file)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 # NEW: Counter for missing body
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        if count > 0:
            logger.info(f"Tracking initialized for {count} targets.")
            messagebox.showinfo("Tracking Updated", f"Now scanning for {count} selected targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging:
                self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Enter seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except Exception: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, bgr_frame)
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # def snap_photo(self):
    #     if self.unprocessed_frame is None: return

    #     # --- FIX START: Force image into dlib-compatible format ---
    #     import numpy as np
    #     # Convert to RGB
    #     rgb_frame = cv2.cvtColor(self.unprocessed_frame, cv2.COLOR_BGR2RGB)
    #     # Force the data to be 8-bit integers and contiguous in memory
    #     rgb_frame = np.ascontiguousarray(rgb_frame, dtype=np.uint8)
    #     # --- FIX END ---

    #     rgb_frame = cv2.cvtColor(self.unprocessed_frame, cv2.COLOR_BGR2RGB)
    #     face_locations = face_recognition.face_locations(rgb_frame)
    #     if len(face_locations) == 1:
    #          name = simpledialog.askstring("Name", "Enter Name:")
    #          if name:
    #              safe_name = name.strip().replace(" ", "_")
    #              cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
    #              self.load_targets()
    #              self.exit_capture_mode()
    #     else:
    #         messagebox.showwarning("Error", "Ensure exactly one face is visible.")
    def snap_photo(self):
        if self.unprocessed_frame is None: 
            return
        
        import numpy as np # Ensure numpy is available
        
        try:
            # 1. Debugging: Print what the camera is actually giving us
            h, w, c = self.unprocessed_frame.shape
            print(f"DEBUG: Camera Frame Shape: {h}x{w}, Channels: {c}, Dtype: {self.unprocessed_frame.dtype}")

            # 2. scrubbing: Force strict standard RGB
            # Convert BGR to RGB
            rgb_frame = cv2.cvtColor(self.unprocessed_frame, cv2.COLOR_BGR2RGB)
            
            # 3. Safety: Remove Alpha channel if present (if 4 channels, keep first 3)
            if len(rgb_frame.shape) == 3 and rgb_frame.shape[2] == 4:
                rgb_frame = rgb_frame[:, :, :3]
            
            # 4. Safety: Force 8-bit Unsigned Integers
            if rgb_frame.dtype != np.uint8:
                rgb_frame = rgb_frame.astype(np.uint8)

            # 5. Safety: Force Contiguous Memory (Deep Copy)
            # This is the most common fix for "Unsupported image type"
            rgb_frame = np.ascontiguousarray(rgb_frame)

            # Now try detection
            face_locations = face_recognition.face_locations(rgb_frame)
            
            if len(face_locations) == 1:
                # Visual feedback (Flash white)
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    # Save the original RAW frame (BGR) to disk, not the modified one
                    cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
                    
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
            elif len(face_locations) == 0:
                messagebox.showwarning("Error", "No face detected.")
            else:
                messagebox.showwarning("Error", "Multiple faces detected.")
                
        except Exception as e:
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process image: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
        self.unprocessed_frame = frame.copy()
        if self.is_in_capture_mode:
            self.process_capture_frame(frame)
        else:
            self.process_tracking_frame_optimized(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # --- Full Fill Resize Logic ---
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame.shape[:2]
                # Maintain aspect ratio
                scale = min(lbl_w/w, lbl_h/h)
                new_w, new_h = int(w*scale), int(h*scale)
                frame_rgb = cv2.resize(frame_rgb, (new_w, new_h))
            
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
        self.root.after(10, self.update_video_feed)

    def process_capture_frame(self, frame):
        h, w = frame.shape[:2]
        cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        return frame

    # --- TRACKING LOGIC ---
    def process_tracking_frame_optimized(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL:
            self.re_detect_counter = 0
        
        rgb_full_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x + w, y + h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection (GREEDY BEST MATCH) - Fixes Target Switching
        untracked_targets = [name for name, s in self.targets_status.items() if not s["visible"]]
        
        if untracked_targets and self.re_detect_counter == 0:
            face_locations = face_recognition.face_locations(rgb_full_frame)
            if face_locations:
                face_encodings = face_recognition.face_encodings(rgb_full_frame, face_locations)
                possible_matches = []
                
                for i, unknown_encoding in enumerate(face_encodings):
                    for name in untracked_targets:
                        target_encoding = self.targets_status[name]["encoding"]
                        dist = face_recognition.face_distance([target_encoding], unknown_encoding)[0]
                        if dist < 0.55:
                            possible_matches.append((dist, i, name))
                
                possible_matches.sort(key=lambda x: x[0])
                assigned_faces = set()
                assigned_targets = set()
                
                for dist, face_idx, name in possible_matches:
                    if face_idx in assigned_faces or name in assigned_targets: continue
                    
                    assigned_faces.add(face_idx)
                    assigned_targets.add(name)
                    (top, right, bottom, left) = face_locations[face_idx]
                    
                    tracker = cv2.legacy.TrackerCSRT_create()
                    tracker.init(frame, (left, top, right-left, bottom-top))
                    self.targets_status[name]["tracker"] = tracker
                    self.targets_status[name]["face_box"] = (left, top, right, bottom)
                    self.targets_status[name]["visible"] = True
                    self.targets_status[name]["missing_pose_counter"] = 0

        # 3. Overlap Check (Fixes Merging Targets)
        active_names = [n for n, s in self.targets_status.items() if s["visible"]]
        for i in range(len(active_names)):
            for j in range(i + 1, len(active_names)):
                nameA = active_names[i]
                nameB = active_names[j]
                
                # Check Face Box IoU
                boxA = self.targets_status[nameA]["face_box"]
                boxB = self.targets_status[nameB]["face_box"]
                # Convert to x,y,w,h format for IoU check
                rectA = (boxA[0], boxA[1], boxA[2]-boxA[0], boxA[3]-boxA[1])
                rectB = (boxB[0], boxB[1], boxB[2]-boxB[0], boxB[3]-boxB[1])
                
                iou = calculate_iou(rectA, rectB)
                if iou > 0.5: # Significant overlap
                    # Force re-detection for both
                    self.targets_status[nameA]["tracker"] = None
                    self.targets_status[nameA]["visible"] = False
                    self.targets_status[nameB]["tracker"] = None
                    self.targets_status[nameB]["visible"] = False

        # 4. Processing & Drawing
        required_act = self.required_action_var.get()
        current_time = time.time()

        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                
                # --- CALCULATE BODY BOX (Torso-Centric) ---
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                # Ghost Box Check: Only draw if tracker is confident AND pose is found
                pose_found_in_box = False
                
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        results_crop = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        current_action = "Unknown"
                        if results_crop.pose_landmarks:
                            pose_found_in_box = True
                            status["missing_pose_counter"] = 0 # Reset
                            
                            draw_styled_landmarks(crop, results_crop)
                            raw_action = classify_action(results_crop.pose_landmarks.landmark, (by2-by1), (bx2-bx1))
                            
                            status["pose_buffer"].append(raw_action)
                            if len(status["pose_buffer"]) >= 8:
                                most_common = Counter(status["pose_buffer"]).most_common(1)[0][0]
                                current_action = most_common
                            else:
                                current_action = raw_action

                            if current_action == required_act:
                                if self.is_alert_mode:
                                    status["last_wave_time"] = current_time
                                    status["alert_triggered_state"] = False
                                if self.is_logging and status["last_logged_action"] != required_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, current_action, "SAFE (Reset)", "N/A"))
                                    status["last_logged_action"] = required_act
                            elif status["last_logged_action"] == required_act:
                                status["last_logged_action"] = None
                            
                            # Draw Box ONLY if pose found
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {current_action}", (bx1, by1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Ghost Box Removal Logic
                if not pose_found_in_box:
                    status["missing_pose_counter"] += 1
                    # If tracker says visible, but no pose for 5 frames -> Kill Tracker
                    if status["missing_pose_counter"] > 5:
                        status["tracker"] = None
                        status["visible"] = False

            # Alert Logic
            if self.is_alert_mode:
                time_diff = current_time - status["last_wave_time"]
                time_left = max(0, self.alert_interval - time_diff)
                y_offset = 50 + (list(self.targets_status.keys()).index(name) * 30)
                color = (0, 255, 0) if time_left > 3 else (0, 0, 255)
                
                # Only show status on screen if target is genuinely lost or safe
                status_txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({status_txt}): {time_left:.1f}s", (frame_w - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                if time_diff > self.alert_interval:
                    if (current_time - status["alert_cooldown"]) > 2.5:
                        play_siren_sound()
                        status["alert_cooldown"] = current_time
                        
                        img_path = "N/A"
                        if status["visible"]:
                            # Snapshot logic (Body box re-calc)
                            fx1, fy1, fx2, fy2 = status["face_box"]
                            face_w = fx2 - fx1
                            bx1 = max(0, int(fx1 + (face_w//2) - (face_w * 3)))
                            bx2 = min(frame_w, int(fx1 + (face_w//2) + (face_w * 3)))
                            by1 = max(0, int(fy1 - (face_w * 0.5)))
                            by2 = frame_h
                            if bx1 < bx2:
                                img_path = self.capture_alert_snapshot(frame[by1:by2, bx1:bx2], name)
                        else:
                            img_path = self.capture_alert_snapshot(frame, name)

                        if self.is_logging:
                            log_s = "ALERT CONTINUED" if status["alert_triggered_state"] else "ALERT TRIGGERED"
                            log_a = current_action if status["visible"] else "MISSING"
                            self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, log_a, log_s, img_path))
                            status["alert_triggered_state"] = True

        return frame 

if __name__ == "__main__":
    app = PoseApp()

2025-11-19 15:45:52,819 - INFO - MediaPipe Holistic Loaded.
2025-11-19 15:45:52,823 - INFO - Loading targets...
2025-11-19 15:46:16,818 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: Camera Frame Shape: 480x640, Channels: 3, Dtype: uint8


2025-11-19 15:46:20,245 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: Camera Frame Shape: 480x640, Channels: 3, Dtype: uint8


In [3]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
import warnings
from datetime import datetime
from collections import deque, Counter

# --- 0. Suppress Warnings ---
# Hides the pkg_resources warning from face_recognition
warnings.filterwarnings("ignore", category=UserWarning, module='face_recognition_models')

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
def play_siren_sound():
    def _sound_worker():
        sys_plat = platform.system()
        try:
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- classify_action ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        L_ANKLE = mp_holistic.PoseLandmark.LEFT_ANKLE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]
        l_ankle = landmarks[L_ANKLE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        # 1. Wave Detection
        if l_wrist.visibility > 0.5 and lw_y < nose_y:
            return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y:
            return "Wave Right"
            
        # 2. Sit/Stand Detection
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: # Thigh is horizontal
                return "Sit"
            else:
                return "Standing"

        return "Standing" 

    except Exception as e:
        return "Unknown"

# --- Helper: IoU for Overlap Check ---
def calculate_iou(boxA, boxB):
    # box = (x, y, w, h) -> convert to (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-5)
    return iou

# --- Tkinter Application Class ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Multi-Target)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.RESIZE_SCALE = 1.0 
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            logger.info("MediaPipe Holistic Loaded.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Holistic Model: {e}")
            self.root.destroy()
            return

        self.frame_timestamp_ms = 0 

        # --- Layout ---
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        # 1. Red Zone
        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        # Bottom Container
        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        # 2. Yellow Zone
        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        # 3. Green Zone
        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Widgets
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')

        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets to Track (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.mainloop()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        self.target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in self.target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 4:
                    display_name = " ".join(parts[1:-2])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except Exception as e:
                logger.error(f"Error parsing {f}: {e}")

        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children():
            widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections:
            tk.Label(self.preview_display, text="No Selection", bg="black", fg="white").pack(expand=True)
            return
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    target_h = 130 if len(display_idx) > 1 else 260
                    target_w = 180 if len(display_idx) > 1 else 360
                    h, w = img.shape[:2]
                    scale = min(target_w/w, target_h/h)
                    new_w, new_h = int(w*scale), int(h*scale)
                    img = cv2.resize(img, (new_w, new_h))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except Exception: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections:
            messagebox.showwarning("Selection", "No targets selected.")
            return
        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    target_image_file = face_recognition.load_image_file(filename)
                    encodings = face_recognition.face_encodings(target_image_file)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 # NEW: Counter for missing body
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        if count > 0:
            logger.info(f"Tracking initialized for {count} targets.")
            messagebox.showinfo("Tracking Updated", f"Now scanning for {count} selected targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging:
                self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Enter seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except Exception: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, bgr_frame)
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # --- CORRECTED SNAP_PHOTO FUNCTION ---
    def snap_photo(self):
        if self.unprocessed_frame is None: 
            return
        
        try:
            # 1. Scrub the image: Force standard RGB
            rgb_frame = cv2.cvtColor(self.unprocessed_frame, cv2.COLOR_BGR2RGB)
            
            # 2. Safety: Force 8-bit Unsigned Integers
            if rgb_frame.dtype != np.uint8:
                rgb_frame = rgb_frame.astype(np.uint8)

            # 3. CRITICAL FIX: Force Contiguous Memory
            # This solves the "Unsupported image type" error even if shape/dtype look correct
            rgb_frame = np.ascontiguousarray(rgb_frame)

            # Now try detection
            face_locations = face_recognition.face_locations(rgb_frame)
            
            if len(face_locations) == 1:
                # Visual feedback (Flash white)
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    # Save the original RAW frame (BGR) to disk
                    cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
                    
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
            elif len(face_locations) == 0:
                messagebox.showwarning("Error", "No face detected.")
            else:
                messagebox.showwarning("Error", "Multiple faces detected.")
                
        except Exception as e:
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process image: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
        self.unprocessed_frame = frame.copy()
        if self.is_in_capture_mode:
            self.process_capture_frame(frame)
        else:
            self.process_tracking_frame_optimized(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # --- Full Fill Resize Logic ---
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame.shape[:2]
                # Maintain aspect ratio
                scale = min(lbl_w/w, lbl_h/h)
                new_w, new_h = int(w*scale), int(h*scale)
                frame_rgb = cv2.resize(frame_rgb, (new_w, new_h))
            
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
        self.root.after(10, self.update_video_feed)

    def process_capture_frame(self, frame):
        h, w = frame.shape[:2]
        cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        return frame

    # --- TRACKING LOGIC ---
    def process_tracking_frame_optimized(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL:
            self.re_detect_counter = 0
        
        # SAFE CONVERSION for Tracking
        rgb_full_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_full_frame = np.ascontiguousarray(rgb_full_frame) # Added safety here too
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x + w, y + h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection (GREEDY BEST MATCH) - Fixes Target Switching
        untracked_targets = [name for name, s in self.targets_status.items() if not s["visible"]]
        
        if untracked_targets and self.re_detect_counter == 0:
            face_locations = face_recognition.face_locations(rgb_full_frame)
            if face_locations:
                face_encodings = face_recognition.face_encodings(rgb_full_frame, face_locations)
                possible_matches = []
                
                for i, unknown_encoding in enumerate(face_encodings):
                    for name in untracked_targets:
                        target_encoding = self.targets_status[name]["encoding"]
                        dist = face_recognition.face_distance([target_encoding], unknown_encoding)[0]
                        if dist < 0.55:
                            possible_matches.append((dist, i, name))
                
                possible_matches.sort(key=lambda x: x[0])
                assigned_faces = set()
                assigned_targets = set()
                
                for dist, face_idx, name in possible_matches:
                    if face_idx in assigned_faces or name in assigned_targets: continue
                    
                    assigned_faces.add(face_idx)
                    assigned_targets.add(name)
                    (top, right, bottom, left) = face_locations[face_idx]
                    
                    tracker = cv2.legacy.TrackerCSRT_create()
                    tracker.init(frame, (left, top, right-left, bottom-top))
                    self.targets_status[name]["tracker"] = tracker
                    self.targets_status[name]["face_box"] = (left, top, right, bottom)
                    self.targets_status[name]["visible"] = True
                    self.targets_status[name]["missing_pose_counter"] = 0

        # 3. Overlap Check (Fixes Merging Targets)
        active_names = [n for n, s in self.targets_status.items() if s["visible"]]
        for i in range(len(active_names)):
            for j in range(i + 1, len(active_names)):
                nameA = active_names[i]
                nameB = active_names[j]
                
                # Check Face Box IoU
                boxA = self.targets_status[nameA]["face_box"]
                boxB = self.targets_status[nameB]["face_box"]
                # Convert to x,y,w,h format for IoU check
                rectA = (boxA[0], boxA[1], boxA[2]-boxA[0], boxA[3]-boxA[1])
                rectB = (boxB[0], boxB[1], boxB[2]-boxB[0], boxB[3]-boxB[1])
                
                iou = calculate_iou(rectA, rectB)
                if iou > 0.5: # Significant overlap
                    # Force re-detection for both
                    self.targets_status[nameA]["tracker"] = None
                    self.targets_status[nameA]["visible"] = False
                    self.targets_status[nameB]["tracker"] = None
                    self.targets_status[nameB]["visible"] = False

        # 4. Processing & Drawing
        required_act = self.required_action_var.get()
        current_time = time.time()

        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                
                # --- CALCULATE BODY BOX (Torso-Centric) ---
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                # Ghost Box Check: Only draw if tracker is confident AND pose is found
                pose_found_in_box = False
                
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        results_crop = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        current_action = "Unknown"
                        if results_crop.pose_landmarks:
                            pose_found_in_box = True
                            status["missing_pose_counter"] = 0 # Reset
                            
                            draw_styled_landmarks(crop, results_crop)
                            raw_action = classify_action(results_crop.pose_landmarks.landmark, (by2-by1), (bx2-bx1))
                            
                            status["pose_buffer"].append(raw_action)
                            if len(status["pose_buffer"]) >= 8:
                                most_common = Counter(status["pose_buffer"]).most_common(1)[0][0]
                                current_action = most_common
                            else:
                                current_action = raw_action

                            if current_action == required_act:
                                if self.is_alert_mode:
                                    status["last_wave_time"] = current_time
                                    status["alert_triggered_state"] = False
                                if self.is_logging and status["last_logged_action"] != required_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, current_action, "SAFE (Reset)", "N/A"))
                                    status["last_logged_action"] = required_act
                            elif status["last_logged_action"] == required_act:
                                status["last_logged_action"] = None
                            
                            # Draw Box ONLY if pose found
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {current_action}", (bx1, by1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Ghost Box Removal Logic
                if not pose_found_in_box:
                    status["missing_pose_counter"] += 1
                    # If tracker says visible, but no pose for 5 frames -> Kill Tracker
                    if status["missing_pose_counter"] > 5:
                        status["tracker"] = None
                        status["visible"] = False

            # Alert Logic
            if self.is_alert_mode:
                time_diff = current_time - status["last_wave_time"]
                time_left = max(0, self.alert_interval - time_diff)
                y_offset = 50 + (list(self.targets_status.keys()).index(name) * 30)
                color = (0, 255, 0) if time_left > 3 else (0, 0, 255)
                
                # Only show status on screen if target is genuinely lost or safe
                status_txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({status_txt}): {time_left:.1f}s", (frame_w - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                if time_diff > self.alert_interval:
                    if (current_time - status["alert_cooldown"]) > 2.5:
                        play_siren_sound()
                        status["alert_cooldown"] = current_time
                        
                        img_path = "N/A"
                        if status["visible"]:
                            # Snapshot logic (Body box re-calc)
                            fx1, fy1, fx2, fy2 = status["face_box"]
                            face_w = fx2 - fx1
                            bx1 = max(0, int(fx1 + (face_w//2) - (face_w * 3)))
                            bx2 = min(frame_w, int(fx1 + (face_w//2) + (face_w * 3)))
                            by1 = max(0, int(fy1 - (face_w * 0.5)))
                            by2 = frame_h
                            if bx1 < bx2:
                                img_path = self.capture_alert_snapshot(frame[by1:by2, bx1:bx2], name)
                        else:
                            img_path = self.capture_alert_snapshot(frame, name)

                        if self.is_logging:
                            log_s = "ALERT CONTINUED" if status["alert_triggered_state"] else "ALERT TRIGGERED"
                            log_a = current_action if status["visible"] else "MISSING"
                            self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, log_a, log_s, img_path))
                            status["alert_triggered_state"] = True

        return frame 

if __name__ == "__main__":
    app = PoseApp()

2025-11-19 15:50:53,446 - INFO - MediaPipe Holistic Loaded.
2025-11-19 15:50:53,449 - INFO - Loading targets...
2025-11-19 15:51:05,694 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.
2025-11-19 15:51:12,854 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


In [4]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
is_sound_playing = False # Global flag to prevent overlapping sounds

def play_siren_sound():
    global is_sound_playing
    if is_sound_playing:
        return

    def _sound_worker():
        global is_sound_playing
        is_sound_playing = True
        sys_plat = platform.system()
        try:
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                # MacOS / Linux
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")
        finally:
            is_sound_playing = False

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- classify_action ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        # 1. Wave Detection (Wrist higher than nose)
        if l_wrist.visibility > 0.5 and lw_y < nose_y:
            return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y:
            return "Wave Right"
            
        # 2. Sit/Stand Detection
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            # Check vertical distance between hip and knee
            if abs(l_knee.y - l_hip.y) < 0.15: 
                return "Sit"
            else:
                return "Standing"

        return "Standing" 

    except Exception as e:
        return "Unknown"

# --- Helper: IoU for Overlap Check ---
def calculate_iou(boxA, boxB):
    # box = (x, y, w, h) -> convert to (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-5)
    return iou

# --- Tracker Factory ---
def create_tracker():
    """Creates a CSRT tracker handling different OpenCV versions."""
    try:
        # Try legacy API (OpenCV 4.5.x with contrib)
        return cv2.legacy.TrackerCSRT_create()
    except AttributeError:
        try:
            # Try modern API (OpenCV 4.10+)
            return cv2.TrackerCSRT_create()
        except AttributeError:
            print("Error: CSRT Tracker not found. Ensure opencv-contrib-python is installed.")
            return None

# --- Tkinter Application Class ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Multi-Target)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            logger.info("MediaPipe Holistic Loaded.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Holistic Model: {e}")
            self.root.destroy()
            return

        # --- Layout ---
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        # 1. Red Zone (Video Feed)
        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        # Bottom Container
        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        # 2. Yellow Zone (Controls)
        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        # 3. Green Zone (Previews)
        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Widgets
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')

        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets to Track (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def on_close(self):
        self.stop_camera()
        self.root.destroy()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 3: # target_Name_face.jpg
                    display_name = " ".join(parts[1:-1])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except Exception as e:
                logger.error(f"Error parsing {f}: {e}")

        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children():
            widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections:
            tk.Label(self.preview_display, text="No Selection", bg="black", fg="white").pack(expand=True)
            return
        
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    target_h = 130 if len(display_idx) > 1 else 260
                    target_w = 180 if len(display_idx) > 1 else 360
                    h, w = img.shape[:2]
                    scale = min(target_w/w, target_h/h)
                    new_w, new_h = int(w*scale), int(h*scale)
                    img = cv2.resize(img, (new_w, new_h))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except Exception: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections:
            messagebox.showwarning("Selection", "No targets selected.")
            return
            
        # Check if "No targets found" is selected
        if self.target_listbox.get(selections[0]) == "No targets found":
            return

        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    target_image_file = face_recognition.load_image_file(filename)
                    encodings = face_recognition.face_encodings(target_image_file)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        if count > 0:
            logger.info(f"Tracking initialized for {count} targets.")
            messagebox.showinfo("Tracking Updated", f"Now scanning for {count} selected targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging:
                self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Enter seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
                self.targets_status[name]["alert_triggered_state"] = False

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except Exception: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            # Ensure we write BGR format to disk
            if len(frame.shape) == 3 and frame.shape[2] == 3:
                # Assuming input frame is BGR from opencv
                cv2.imwrite(filename, frame)
            else:
                cv2.imwrite(filename, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # --- KEY FIX FOR SNAPSHOT ERROR ---
    def snap_photo(self):
        if self.unprocessed_frame is None: 
            return
        
        try:
            # 1. Convert BGR to RGB
            rgb_frame = cv2.cvtColor(self.unprocessed_frame, cv2.COLOR_BGR2RGB)
            
            # 2. Ensure Strict uint8 type
            if rgb_frame.dtype != np.uint8:
                rgb_frame = rgb_frame.astype(np.uint8)
            
            # 3. CRITICAL FIX: Ensure memory is Contiguous (C-style)
            # This solves "Unsupported image type" in dlib/face_recognition
            rgb_frame = np.ascontiguousarray(rgb_frame)

            # 4. Detect
            face_locations = face_recognition.face_locations(rgb_frame)
            
            if len(face_locations) == 1:
                # Visual Flash Effect
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    # Save the original raw BGR frame to disk
                    cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
                    
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
            elif len(face_locations) == 0:
                messagebox.showwarning("Error", "No face detected.")
            else:
                messagebox.showwarning("Error", "Multiple faces detected. Only one allowed.")
                
        except Exception as e:
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process image: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
            
        self.unprocessed_frame = frame.copy()
        
        if self.is_in_capture_mode:
            display_frame = self.process_capture_frame(frame)
        else:
            display_frame = self.process_tracking_frame_optimized(frame)
        
        if self.video_label.winfo_exists():
            # Convert to RGB for Tkinter Display
            frame_rgb = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame_rgb.shape[:2]
                scale = min(lbl_w/w, lbl_h/h)
                new_w, new_h = int(w*scale), int(h*scale)
                frame_rgb = cv2.resize(frame_rgb, (new_w, new_h))
            
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
            
        self.root.after(10, self.update_video_feed)

    def process_capture_frame(self, frame):
        h, w = frame.shape[:2]
        cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        return frame

    # --- TRACKING LOGIC ---
    def process_tracking_frame_optimized(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL:
            self.re_detect_counter = 0
        
        # Ensure contiguous RGB frame for face recognition scanning
        rgb_full_frame = np.ascontiguousarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Existing Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x + w, y + h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection (Scanning for lost targets)
        untracked_targets = [name for name, s in self.targets_status.items() if not s["visible"]]
        
        if untracked_targets and self.re_detect_counter == 0:
            face_locations = face_recognition.face_locations(rgb_full_frame)
            if face_locations:
                face_encodings = face_recognition.face_encodings(rgb_full_frame, face_locations)
                possible_matches = []
                
                for i, unknown_encoding in enumerate(face_encodings):
                    for name in untracked_targets:
                        target_encoding = self.targets_status[name]["encoding"]
                        dist = face_recognition.face_distance([target_encoding], unknown_encoding)[0]
                        if dist < 0.55:
                            possible_matches.append((dist, i, name))
                
                possible_matches.sort(key=lambda x: x[0])
                assigned_faces = set()
                assigned_targets = set()
                
                for dist, face_idx, name in possible_matches:
                    if face_idx in assigned_faces or name in assigned_targets: continue
                    
                    assigned_faces.add(face_idx)
                    assigned_targets.add(name)
                    (top, right, bottom, left) = face_locations[face_idx]
                    
                    tracker = create_tracker()
                    if tracker:
                        tracker.init(frame, (left, top, right-left, bottom-top))
                        self.targets_status[name]["tracker"] = tracker
                        self.targets_status[name]["face_box"] = (left, top, right, bottom)
                        self.targets_status[name]["visible"] = True
                        self.targets_status[name]["missing_pose_counter"] = 0

        # 3. Overlap Check (Prevent merging targets)
        active_names = [n for n, s in self.targets_status.items() if s["visible"]]
        for i in range(len(active_names)):
            for j in range(i + 1, len(active_names)):
                nameA = active_names[i]
                nameB = active_names[j]
                
                boxA = self.targets_status[nameA]["face_box"]
                boxB = self.targets_status[nameB]["face_box"]
                
                rectA = (boxA[0], boxA[1], boxA[2]-boxA[0], boxA[3]-boxA[1])
                rectB = (boxB[0], boxB[1], boxB[2]-boxB[0], boxB[3]-boxB[1])
                
                if calculate_iou(rectA, rectB) > 0.5:
                    self.targets_status[nameA]["tracker"] = None
                    self.targets_status[nameA]["visible"] = False
                    self.targets_status[nameB]["tracker"] = None
                    self.targets_status[nameB]["visible"] = False

        # 4. Pose Analysis & Drawing
        required_act = self.required_action_var.get()
        current_time = time.time()

        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                
                # Estimate Body Box relative to Face
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                pose_found_in_box = False
                
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        # Prepare crop for MediaPipe (RGB + Not Writeable = faster)
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        results_crop = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        current_action = "Unknown"
                        if results_crop.pose_landmarks:
                            pose_found_in_box = True
                            status["missing_pose_counter"] = 0 
                            
                            draw_styled_landmarks(crop, results_crop)
                            raw_action = classify_action(results_crop.pose_landmarks.landmark, (by2-by1), (bx2-bx1))
                            
                            status["pose_buffer"].append(raw_action)
                            if len(status["pose_buffer"]) >= 8:
                                most_common = Counter(status["pose_buffer"]).most_common(1)[0][0]
                                current_action = most_common
                            else:
                                current_action = raw_action

                            # Action Logic
                            if current_action == required_act:
                                if self.is_alert_mode:
                                    status["last_wave_time"] = current_time
                                    status["alert_triggered_state"] = False
                                if self.is_logging and status["last_logged_action"] != required_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, current_action, "SAFE (Reset)", "N/A"))
                                    status["last_logged_action"] = required_act
                            elif status["last_logged_action"] == required_act:
                                status["last_logged_action"] = None
                            
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {current_action}", (bx1, by1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Ghost Box Check
                if not pose_found_in_box:
                    status["missing_pose_counter"] += 1
                    if status["missing_pose_counter"] > 5:
                        status["tracker"] = None
                        status["visible"] = False

            # Alert Logic
            if self.is_alert_mode:
                time_diff = current_time - status["last_wave_time"]
                time_left = max(0, self.alert_interval - time_diff)
                y_offset = 50 + (list(self.targets_status.keys()).index(name) * 30)
                color = (0, 255, 0) if time_left > 3 else (0, 0, 255)
                
                status_txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({status_txt}): {time_left:.1f}s", (frame_w - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                if time_diff > self.alert_interval:
                    if (current_time - status["alert_cooldown"]) > 2.5:
                        play_siren_sound()
                        status["alert_cooldown"] = current_time
                        
                        img_path = "N/A"
                        # Capture logic
                        if status["visible"]:
                            fx1, fy1, fx2, fy2 = status["face_box"]
                            face_w = fx2 - fx1
                            bx1 = max(0, int(fx1 + (face_w//2) - (face_w * 3)))
                            bx2 = min(frame_w, int(fx1 + (face_w//2) + (face_w * 3)))
                            by1 = max(0, int(fy1 - (face_w * 0.5)))
                            by2 = frame_h
                            if bx1 < bx2:
                                img_path = self.capture_alert_snapshot(frame[by1:by2, bx1:bx2], name)
                        else:
                            img_path = self.capture_alert_snapshot(frame, name)

                        if self.is_logging:
                            log_s = "ALERT CONTINUED" if status["alert_triggered_state"] else "ALERT TRIGGERED"
                            log_a = current_action if status.get("visible") else "MISSING"
                            self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, log_a, log_s, img_path))
                            status["alert_triggered_state"] = True

        return frame 

if __name__ == "__main__":
    app = PoseApp()

2025-11-19 15:52:24,568 - INFO - MediaPipe Holistic Loaded.
2025-11-19 15:52:24,573 - INFO - Loading targets...
2025-11-19 15:52:43,165 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.
2025-11-19 15:52:47,460 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


In [5]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

# Ensure directories exist
if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic (Thread Safe) ---
is_sound_playing = False 

def play_siren_sound():
    global is_sound_playing
    if is_sound_playing: return

    def _sound_worker():
        global is_sound_playing
        is_sound_playing = True
        try:
            sys_plat = platform.system()
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")
        finally:
            is_sound_playing = False

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- Action Classification ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        if l_wrist.visibility > 0.5 and lw_y < nose_y: return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y: return "Wave Right"
        
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: return "Sit"
            else: return "Standing"

        return "Standing" 
    except: return "Unknown"

def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea + 1e-5)

def create_tracker():
    try: return cv2.legacy.TrackerCSRT_create()
    except: 
        try: return cv2.TrackerCSRT_create()
        except: return None

# --- THE FIX: Helper to Sanitize Images for Dlib ---
def _sanitize_for_dlib(cv2_frame):
    """
    Converts an OpenCV image (BGR) to a Dlib-compatible RGB image.
    Ensures 8-bit, 3-channel, C-Contiguous memory layout.
    """
    try:
        # 1. Convert BGR to RGB
        rgb = cv2.cvtColor(cv2_frame, cv2.COLOR_BGR2RGB)
        
        # 2. Strip Alpha channel if present (e.g. 4 channels)
        if rgb.shape[2] == 4:
            rgb = rgb[:, :, :3]
            
        # 3. Force 8-bit unsigned integer
        if rgb.dtype != np.uint8:
            rgb = rgb.astype(np.uint8)
            
        # 4. CRITICAL: Force contiguous memory layout
        rgb = np.ascontiguousarray(rgb)
        
        return rgb
    except Exception as e:
        logger.error(f"Sanitization failed: {e}")
        return None

# --- Tkinter Application ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Stable)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        except Exception as e:
            messagebox.showerror("Error", f"MediaPipe Error: {e}")
            self.root.destroy()
            return

        # UI Layout
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Controls
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')
        
        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def on_close(self):
        self.stop_camera()
        self.root.destroy()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 3:
                    display_name = " ".join(parts[1:-1])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except: pass
        
        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children(): widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections: return
        
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (180, 130))
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections: return
        
        if self.target_listbox.get(selections[0]) == "No targets found": return

        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    # Use sanitization here too for safety
                    raw_img = cv2.imread(filename)
                    sanitized_img = _sanitize_for_dlib(raw_img)
                    if sanitized_img is None: continue
                    
                    encodings = face_recognition.face_encodings(sanitized_img)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        
        if count > 0:
            messagebox.showinfo("Tracking Updated", f"Scanning for {count} targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging: self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            cv2.imwrite(filename, frame) # frame is already BGR
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # --- THIS IS THE FIXED SNAP METHOD ---
    def snap_photo(self):
        if self.unprocessed_frame is None: return
        
        try:
            # Use the sanitizer helper (NUCLEAR FIX)
            dlib_ready_image = _sanitize_for_dlib(self.unprocessed_frame)
            
            if dlib_ready_image is None:
                raise ValueError("Image sanitization failed")

            # Detect
            face_locations = face_recognition.face_locations(dlib_ready_image)
            
            if len(face_locations) == 1:
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
            elif len(face_locations) == 0:
                messagebox.showwarning("Error", "No face detected.")
            else:
                messagebox.showwarning("Error", "Multiple faces detected.")
                
        except Exception as e:
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
            
        self.unprocessed_frame = frame.copy()
        
        if self.is_in_capture_mode:
            h, w = frame.shape[:2]
            cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        else:
            frame = self.process_tracking(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame_rgb.shape[:2]
                scale = min(lbl_w/w, lbl_h/h)
                frame_rgb = cv2.resize(frame_rgb, (int(w*scale), int(h*scale)))
            
            imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
            
        self.root.after(10, self.update_video_feed)

    def process_tracking(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL: self.re_detect_counter = 0
        
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x+w, y+h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection
        untracked_targets = [n for n, s in self.targets_status.items() if not s["visible"]]
        if untracked_targets and self.re_detect_counter == 0:
            # Use the sanitizer helper here too (NUCLEAR FIX)
            clean_frame = _sanitize_for_dlib(frame)
            
            if clean_frame is not None:
                locs = face_recognition.face_locations(clean_frame)
                if locs:
                    encs = face_recognition.face_encodings(clean_frame, locs)
                    matches = []
                    for i, unk_enc in enumerate(encs):
                        for name in untracked_targets:
                            dist = face_recognition.face_distance([self.targets_status[name]["encoding"]], unk_enc)[0]
                            if dist < 0.55: matches.append((dist, i, name))
                    
                    matches.sort(key=lambda x: x[0])
                    assigned = set()
                    
                    for dist, idx, name in matches:
                        if idx in assigned or self.targets_status[name]["visible"]: continue
                        assigned.add(idx)
                        t, r, b, l = locs[idx]
                        
                        tracker = create_tracker()
                        if tracker:
                            tracker.init(frame, (l, t, r-l, b-t))
                            self.targets_status[name]["tracker"] = tracker
                            self.targets_status[name]["face_box"] = (l, t, r, b)
                            self.targets_status[name]["visible"] = True
                            self.targets_status[name]["missing_pose_counter"] = 0

        # 3. Draw & Pose
        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                
                # Body Box
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                pose_found = False
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        res = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        if res.pose_landmarks:
                            pose_found = True
                            status["missing_pose_counter"] = 0
                            draw_styled_landmarks(crop, res)
                            act = classify_action(res.pose_landmarks.landmark, by2-by1, bx2-bx1)
                            status["pose_buffer"].append(act)
                            
                            curr_act = act
                            if len(status["pose_buffer"]) >= 8:
                                curr_act = Counter(status["pose_buffer"]).most_common(1)[0][0]
                            
                            req_act = self.required_action_var.get()
                            if curr_act == req_act:
                                if self.is_alert_mode: status["last_wave_time"] = time.time()
                                if self.is_logging and status["last_logged_action"] != req_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, curr_act, "SAFE", "N/A"))
                                    status["last_logged_action"] = req_act
                            
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {curr_act}", (bx1, by1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                if not pose_found:
                    status["missing_pose_counter"] += 1
                    if status["missing_pose_counter"] > 10:
                        status["tracker"] = None
                        status["visible"] = False
            
            # Alert Overlay
            if self.is_alert_mode:
                diff = time.time() - status["last_wave_time"]
                rem = max(0, self.alert_interval - diff)
                col = (0,255,0) if rem > 3 else (0,0,255)
                y_off = 50 + (list(self.targets_status.keys()).index(name)*30)
                txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({txt}): {rem:.1f}s", (frame_w-300, y_off), cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)
                
                if diff > self.alert_interval and (time.time() - status["alert_cooldown"] > 2.5):
                    play_siren_sound()
                    status["alert_cooldown"] = time.time()
                    self.capture_alert_snapshot(frame, name)
                    if self.is_logging:
                        self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, "MISSING/IDLE", "ALERT", "CAPTURED"))

        return frame

if __name__ == "__main__":
    app = PoseApp()

2025-11-19 15:56:04,438 - INFO - Loading targets...
2025-11-19 15:56:25,551 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.
2025-11-19 15:56:32,464 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


In [6]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter
import shutil # Added for file operations

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

# Ensure directories exist
if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
is_sound_playing = False 

def play_siren_sound():
    global is_sound_playing
    if is_sound_playing: return

    def _sound_worker():
        global is_sound_playing
        is_sound_playing = True
        try:
            sys_plat = platform.system()
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")
        finally:
            is_sound_playing = False

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- Action Classification ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        if l_wrist.visibility > 0.5 and lw_y < nose_y: return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y: return "Wave Right"
        
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: return "Sit"
            else: return "Standing"

        return "Standing" 
    except: return "Unknown"

def create_tracker():
    try: return cv2.legacy.TrackerCSRT_create()
    except: 
        try: return cv2.TrackerCSRT_create()
        except: return None

# --- Tkinter Application ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Disk IO Safe Mode)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        except Exception as e:
            messagebox.showerror("Error", f"MediaPipe Error: {e}")
            self.root.destroy()
            return

        # UI Layout
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Controls
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')
        
        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def on_close(self):
        self.stop_camera()
        self.root.destroy()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 3:
                    display_name = " ".join(parts[1:-1])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except: pass
        
        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children(): widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections: return
        
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (180, 130))
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections: return
        
        if self.target_listbox.get(selections[0]) == "No targets found": return

        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    # Safe loading from disk (Standard way)
                    encodings = face_recognition.face_encodings(face_recognition.load_image_file(filename))
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        
        if count > 0:
            messagebox.showinfo("Tracking Updated", f"Scanning for {count} targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging: self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            cv2.imwrite(filename, frame)
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # --- 100% SAFE SNAPSHOT METHOD (Disk I/O) ---
    def snap_photo(self):
        if self.unprocessed_frame is None: return
        
        temp_filename = "temp_snap_buffer.jpg"
        try:
            # 1. Save Current Frame to Disk (Cleans memory/format issues)
            cv2.imwrite(temp_filename, self.unprocessed_frame)
            
            # 2. Load it back using face_recognition's own loader
            # This guarantees the image is exactly how the library wants it
            image_from_disk = face_recognition.load_image_file(temp_filename)

            # 3. Detect
            face_locations = face_recognition.face_locations(image_from_disk)
            
            if len(face_locations) == 1:
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    final_filename = f"target_{safe_name}_face.jpg"
                    
                    # Rename temp file to final file (Efficient move)
                    shutil.move(temp_filename, final_filename)
                    
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
                else:
                    os.remove(temp_filename) # Clean up if cancelled
            elif len(face_locations) == 0:
                os.remove(temp_filename)
                messagebox.showwarning("Error", "No face detected.")
            else:
                os.remove(temp_filename)
                messagebox.showwarning("Error", "Multiple faces detected.")
                
        except Exception as e:
            if os.path.exists(temp_filename):
                os.remove(temp_filename)
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
            
        self.unprocessed_frame = frame.copy()
        
        if self.is_in_capture_mode:
            h, w = frame.shape[:2]
            cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        else:
            frame = self.process_tracking(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame_rgb.shape[:2]
                scale = min(lbl_w/w, lbl_h/h)
                frame_rgb = cv2.resize(frame_rgb, (int(w*scale), int(h*scale)))
            
            imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
            
        self.root.after(10, self.update_video_feed)

    def process_tracking(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL: self.re_detect_counter = 0
        
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x+w, y+h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection
        untracked_targets = [n for n, s in self.targets_status.items() if not s["visible"]]
        if untracked_targets and self.re_detect_counter == 0:
            # We try standard detection. If this crashes, we might need to suppress it or use disk approach here too.
            # For now, let's assume the crash is mostly on the manual snapshot based on logs.
            try:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                locs = face_recognition.face_locations(rgb_frame)
                if locs:
                    encs = face_recognition.face_encodings(rgb_frame, locs)
                    matches = []
                    for i, unk_enc in enumerate(encs):
                        for name in untracked_targets:
                            dist = face_recognition.face_distance([self.targets_status[name]["encoding"]], unk_enc)[0]
                            if dist < 0.55: matches.append((dist, i, name))
                    
                    matches.sort(key=lambda x: x[0])
                    assigned = set()
                    
                    for dist, idx, name in matches:
                        if idx in assigned or self.targets_status[name]["visible"]: continue
                        assigned.add(idx)
                        t, r, b, l = locs[idx]
                        
                        tracker = create_tracker()
                        if tracker:
                            tracker.init(frame, (l, t, r-l, b-t))
                            self.targets_status[name]["tracker"] = tracker
                            self.targets_status[name]["face_box"] = (l, t, r, b)
                            self.targets_status[name]["visible"] = True
                            self.targets_status[name]["missing_pose_counter"] = 0
            except Exception:
                pass # Silently fail detection frame to avoid crashing main loop

        # 3. Draw & Pose
        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                
                # Body Box
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                pose_found = False
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        res = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        if res.pose_landmarks:
                            pose_found = True
                            status["missing_pose_counter"] = 0
                            draw_styled_landmarks(crop, res)
                            act = classify_action(res.pose_landmarks.landmark, by2-by1, bx2-bx1)
                            status["pose_buffer"].append(act)
                            
                            curr_act = act
                            if len(status["pose_buffer"]) >= 8:
                                curr_act = Counter(status["pose_buffer"]).most_common(1)[0][0]
                            
                            req_act = self.required_action_var.get()
                            if curr_act == req_act:
                                if self.is_alert_mode: status["last_wave_time"] = time.time()
                                if self.is_logging and status["last_logged_action"] != req_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, curr_act, "SAFE", "N/A"))
                                    status["last_logged_action"] = req_act
                            
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {curr_act}", (bx1, by1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                if not pose_found:
                    status["missing_pose_counter"] += 1
                    if status["missing_pose_counter"] > 10:
                        status["tracker"] = None
                        status["visible"] = False
            
            # Alert Overlay
            if self.is_alert_mode:
                diff = time.time() - status["last_wave_time"]
                rem = max(0, self.alert_interval - diff)
                col = (0,255,0) if rem > 3 else (0,0,255)
                y_off = 50 + (list(self.targets_status.keys()).index(name)*30)
                txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({txt}): {rem:.1f}s", (frame_w-300, y_off), cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)
                
                if diff > self.alert_interval and (time.time() - status["alert_cooldown"] > 2.5):
                    play_siren_sound()
                    status["alert_cooldown"] = time.time()
                    self.capture_alert_snapshot(frame, name)
                    if self.is_logging:
                        self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, "MISSING/IDLE", "ALERT", "CAPTURED"))

        return frame

if __name__ == "__main__":
    app = PoseApp()

2025-11-19 16:02:08,920 - INFO - Loading targets...
2025-11-19 16:02:22,220 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.
2025-11-19 16:02:25,428 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.
2025-11-19 16:02:29,365 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


In [7]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter
import shutil 

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
is_sound_playing = False 

def play_siren_sound():
    global is_sound_playing
    if is_sound_playing: return

    def _sound_worker():
        global is_sound_playing
        is_sound_playing = True
        try:
            sys_plat = platform.system()
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")
        finally:
            is_sound_playing = False

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- Action Classification ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        if l_wrist.visibility > 0.5 and lw_y < nose_y: return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y: return "Wave Right"
        
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: return "Sit"
            else: return "Standing"

        return "Standing" 
    except: return "Unknown"

def create_tracker():
    try: return cv2.legacy.TrackerCSRT_create()
    except: 
        try: return cv2.TrackerCSRT_create()
        except: return None

# --- Tkinter Application ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Production Fix)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        except Exception as e:
            messagebox.showerror("Error", f"MediaPipe Error: {e}")
            self.root.destroy()
            return

        # UI Layout
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Controls
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')
        
        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def on_close(self):
        self.stop_camera()
        self.root.destroy()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 3:
                    display_name = " ".join(parts[1:-1])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except: pass
        
        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children(): widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections: return
        
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (180, 130))
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections: return
        
        if self.target_listbox.get(selections[0]) == "No targets found": return

        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    # Safe loading: OpenCV -> RGB -> uint8 -> Contiguous
                    raw_img = cv2.imread(filename)
                    if raw_img is None: continue
                    
                    rgb_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)
                    # Ensure strictly 3 channels, 8-bit, contiguous
                    rgb_img = np.ascontiguousarray(rgb_img[:, :, :3]).astype(np.uint8)

                    encodings = face_recognition.face_encodings(rgb_img)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        
        if count > 0:
            messagebox.showinfo("Tracking Updated", f"Scanning for {count} targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging: self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            cv2.imwrite(filename, frame)
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # --- 100% NUCLEAR FIX FOR SNAPSHOT ERROR ---
    def snap_photo(self):
        if self.unprocessed_frame is None: return
        
        temp_filename = "temp_snap_buffer.jpg"
        
        try:
            # 1. Save the raw BGR frame to disk (This cleanses memory garbage)
            cv2.imwrite(temp_filename, self.unprocessed_frame)
            
            # 2. Load back using OpenCV (NOT face_recognition's loader, which uses PIL)
            loaded_img = cv2.imread(temp_filename)
            
            if loaded_img is None:
                raise ValueError("Could not read back temporary image.")

            # 3. Manual Strict Conversion to RGB
            rgb_img = cv2.cvtColor(loaded_img, cv2.COLOR_BGR2RGB)
            
            # 4. Force Data Types for Dlib (8-bit Unsigned Int)
            rgb_img = rgb_img.astype(np.uint8)
            
            # 5. Force C-Contiguous Memory (The most likely culprit)
            rgb_img = np.ascontiguousarray(rgb_img)
            
            # 6. Debug Print to console if it fails
            print(f"DEBUG: Snapshot Shape: {rgb_img.shape}, Dtype: {rgb_img.dtype}, Contiguous: {rgb_img.flags['C_CONTIGUOUS']}")

            # 7. Detect
            face_locations = face_recognition.face_locations(rgb_img)
            
            if len(face_locations) == 1:
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    final_filename = f"target_{safe_name}_face.jpg"
                    shutil.move(temp_filename, final_filename)
                    
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
                else:
                    if os.path.exists(temp_filename): os.remove(temp_filename)
            elif len(face_locations) == 0:
                if os.path.exists(temp_filename): os.remove(temp_filename)
                messagebox.showwarning("Error", "No face detected.")
            else:
                if os.path.exists(temp_filename): os.remove(temp_filename)
                messagebox.showwarning("Error", "Multiple faces detected.")
                
        except Exception as e:
            if os.path.exists(temp_filename): os.remove(temp_filename)
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
            
        self.unprocessed_frame = frame.copy()
        
        if self.is_in_capture_mode:
            h, w = frame.shape[:2]
            cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        else:
            frame = self.process_tracking(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame_rgb.shape[:2]
                scale = min(lbl_w/w, lbl_h/h)
                frame_rgb = cv2.resize(frame_rgb, (int(w*scale), int(h*scale)))
            
            imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
            
        self.root.after(10, self.update_video_feed)

    def process_tracking(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL: self.re_detect_counter = 0
        
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x+w, y+h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection
        untracked_targets = [n for n, s in self.targets_status.items() if not s["visible"]]
        if untracked_targets and self.re_detect_counter == 0:
            # Use the same SAFE LOADING method for detection loop
            try:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Ensure strict format for detection too
                rgb_frame = np.ascontiguousarray(rgb_frame).astype(np.uint8)
                
                locs = face_recognition.face_locations(rgb_frame)
                if locs:
                    encs = face_recognition.face_encodings(rgb_frame, locs)
                    matches = []
                    for i, unk_enc in enumerate(encs):
                        for name in untracked_targets:
                            dist = face_recognition.face_distance([self.targets_status[name]["encoding"]], unk_enc)[0]
                            if dist < 0.55: matches.append((dist, i, name))
                    
                    matches.sort(key=lambda x: x[0])
                    assigned = set()
                    
                    for dist, idx, name in matches:
                        if idx in assigned or self.targets_status[name]["visible"]: continue
                        assigned.add(idx)
                        t, r, b, l = locs[idx]
                        
                        tracker = create_tracker()
                        if tracker:
                            tracker.init(frame, (l, t, r-l, b-t))
                            self.targets_status[name]["tracker"] = tracker
                            self.targets_status[name]["face_box"] = (l, t, r, b)
                            self.targets_status[name]["visible"] = True
                            self.targets_status[name]["missing_pose_counter"] = 0
            except Exception: pass

        # 3. Draw & Pose
        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                
                # Body Box
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                pose_found = False
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        res = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        if res.pose_landmarks:
                            pose_found = True
                            status["missing_pose_counter"] = 0
                            draw_styled_landmarks(crop, res)
                            act = classify_action(res.pose_landmarks.landmark, by2-by1, bx2-bx1)
                            status["pose_buffer"].append(act)
                            
                            curr_act = act
                            if len(status["pose_buffer"]) >= 8:
                                curr_act = Counter(status["pose_buffer"]).most_common(1)[0][0]
                            
                            req_act = self.required_action_var.get()
                            if curr_act == req_act:
                                if self.is_alert_mode: status["last_wave_time"] = time.time()
                                if self.is_logging and status["last_logged_action"] != req_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, curr_act, "SAFE", "N/A"))
                                    status["last_logged_action"] = req_act
                            
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {curr_act}", (bx1, by1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                if not pose_found:
                    status["missing_pose_counter"] += 1
                    if status["missing_pose_counter"] > 10:
                        status["tracker"] = None
                        status["visible"] = False
            
            # Alert Overlay
            if self.is_alert_mode:
                diff = time.time() - status["last_wave_time"]
                rem = max(0, self.alert_interval - diff)
                col = (0,255,0) if rem > 3 else (0,0,255)
                y_off = 50 + (list(self.targets_status.keys()).index(name)*30)
                txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({txt}): {rem:.1f}s", (frame_w-300, y_off), cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)
                
                if diff > self.alert_interval and (time.time() - status["alert_cooldown"] > 2.5):
                    play_siren_sound()
                    status["alert_cooldown"] = time.time()
                    self.capture_alert_snapshot(frame, name)
                    if self.is_logging:
                        self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, "MISSING/IDLE", "ALERT", "CAPTURED"))

        return frame

if __name__ == "__main__":
    app = PoseApp()

2025-11-19 16:05:54,651 - INFO - Loading targets...
2025-11-19 16:06:07,371 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: Snapshot Shape: (480, 640, 3), Dtype: uint8, Contiguous: True


2025-11-19 16:06:10,129 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: Snapshot Shape: (480, 640, 3), Dtype: uint8, Contiguous: True


2025-11-19 16:06:15,231 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: Snapshot Shape: (480, 640, 3), Dtype: uint8, Contiguous: True


In [1]:
import cv2
import mediapipe as mp
import csv
import time
import tkinter as tk
from tkinter import font
from tkinter import simpledialog
from tkinter import messagebox 
from PIL import Image, ImageTk
import os 
import glob
import face_recognition
import numpy as np 
import threading 
import platform
import logging
from datetime import datetime
from collections import deque, Counter

# --- 1. Logging Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("PoseGuard")

if not os.path.exists("alert_snapshots"):
    os.makedirs("alert_snapshots")

csv_file = "activity_log.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path"])

# --- MediaPipe Solutions Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# --- Sound Logic ---
is_sound_playing = False 

def play_siren_sound():
    global is_sound_playing
    if is_sound_playing: return

    def _sound_worker():
        global is_sound_playing
        is_sound_playing = True
        try:
            sys_plat = platform.system()
            if sys_plat == "Windows":
                import winsound
                for _ in range(3):
                    winsound.Beep(2000, 300) 
                    winsound.Beep(1000, 300) 
            else:
                for _ in range(3):
                    print('\a')
                    time.sleep(0.3)
        except Exception as e:
            logger.error(f"Sound Error: {e}")
        finally:
            is_sound_playing = False

    t = threading.Thread(target=_sound_worker, daemon=True)
    t.start()

# --- Styled Drawing Helper ---
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,255,121), thickness=1, circle_radius=1)) 
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

# --- Action Classification ---
def classify_action(landmarks, h, w):
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        
        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_hip = landmarks[L_HIP]
        l_knee = landmarks[L_KNEE]

        nose_y = nose.y * h
        lw_y = l_wrist.y * h
        rw_y = r_wrist.y * h
        
        if l_wrist.visibility > 0.5 and lw_y < nose_y: return "Wave Left"
        if r_wrist.visibility > 0.5 and rw_y < nose_y: return "Wave Right"
        
        if l_hip.visibility > 0.5 and l_knee.visibility > 0.5:
            if abs(l_knee.y - l_hip.y) < 0.15: return "Sit"
            else: return "Standing"

        return "Standing" 
    except: return "Unknown"

def create_tracker():
    try: return cv2.legacy.TrackerCSRT_create()
    except: 
        try: return cv2.TrackerCSRT_create()
        except: return None

def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea + 1e-5)

# --- THE FIX: Memory Scrubber ---
def scrub_image_memory(image, to_gray=False):
    """
    Completely recreates the numpy array from raw bytes.
    This bypasses strict stride checks in dlib caused by numpy version mismatches.
    """
    try:
        h, w = image.shape[:2]
        
        if to_gray:
            # Convert to Gray, then Serialize
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image
            
            # Serialize to raw bytes (removes all metadata)
            raw_bytes = gray.tobytes()
            # Reconstruct fresh
            clean_img = np.frombuffer(raw_bytes, dtype=np.uint8).reshape((h, w))
            return clean_img
            
        else:
            # Convert to RGB, then Serialize
            if len(image.shape) == 3:
                # Ensure BGR to RGB
                rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            else:
                rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
                
            # Serialize to raw bytes (removes all metadata)
            raw_bytes = rgb.tobytes()
            # Reconstruct fresh
            clean_img = np.frombuffer(raw_bytes, dtype=np.uint8).reshape((h, w, 3))
            return clean_img

    except Exception as e:
        logger.error(f"Scrubbing failed: {e}")
        return None

# --- Tkinter Application ---
class PoseApp:
    def __init__(self, window_title="Pose Guard (Memory Safe)"):
        self.root = tk.Tk()
        self.root.title(window_title)
        self.root.geometry("1400x950")
        self.root.configure(bg="black") 
        
        self.cap = None
        self.unprocessed_frame = None 
        self.is_running = False
        self.is_logging = False
        
        self.is_alert_mode = False
        self.alert_interval = 10  
        self.is_in_capture_mode = False
        self.frame_w = 640 
        self.frame_h = 480 

        self.target_map = {}
        self.targets_status = {} 
        self.re_detect_counter = 0    
        self.RE_DETECT_INTERVAL = 30  
        self.temp_log = [] 
        
        try:
            self.holistic_full = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
            self.holistic_crop = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        except Exception as e:
            messagebox.showerror("Error", f"MediaPipe Error: {e}")
            self.root.destroy()
            return

        # UI Layout
        self.root.grid_rowconfigure(0, weight=3) 
        self.root.grid_rowconfigure(1, weight=1) 
        self.root.grid_columnconfigure(0, weight=1)

        self.red_zone = tk.Frame(self.root, bg="red", bd=4)
        self.red_zone.grid(row=0, column=0, sticky="nsew", padx=2, pady=2)
        self.video_container = tk.Frame(self.red_zone, bg="black")
        self.video_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.video_label = tk.Label(self.video_container, bg="black", text="Camera Feed Off", fg="white")
        self.video_label.pack(fill="both", expand=True)

        self.bottom_container = tk.Frame(self.root, bg="black")
        self.bottom_container.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        self.bottom_container.grid_columnconfigure(0, weight=7) 
        self.bottom_container.grid_columnconfigure(1, weight=3) 
        self.bottom_container.grid_rowconfigure(0, weight=1)

        self.yellow_zone = tk.Frame(self.bottom_container, bg="gold", bd=4)
        self.yellow_zone.grid(row=0, column=0, sticky="nsew", padx=2)
        self.controls_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.controls_frame.pack(side="top", fill="x", padx=5, pady=5)
        self.listbox_frame = tk.Frame(self.yellow_zone, bg="gold")
        self.listbox_frame.pack(side="top", fill="both", expand=True, padx=5, pady=5)

        self.green_zone = tk.Frame(self.bottom_container, bg="#00FF00", bd=4)
        self.green_zone.grid(row=0, column=1, sticky="nsew", padx=2)
        self.preview_container = tk.Frame(self.green_zone, bg="black")
        self.preview_container.pack(fill="both", expand=True, padx=2, pady=2)
        self.preview_display = tk.Frame(self.preview_container, bg="black")
        self.preview_display.pack(fill="both", expand=True)

        # Controls
        btn_font = font.Font(family='Helvetica', size=10, weight='bold')
        
        self.btn_start = tk.Button(self.controls_frame, text="Start Camera", command=self.start_camera, font=btn_font, bg="#27ae60", fg="white", width=12)
        self.btn_start.grid(row=0, column=0, padx=3, pady=3)
        self.btn_stop = tk.Button(self.controls_frame, text="Stop Camera", command=self.stop_camera, font=btn_font, bg="#c0392b", fg="white", width=12, state="disabled")
        self.btn_stop.grid(row=0, column=1, padx=3, pady=3)
        self.btn_toggle_log = tk.Button(self.controls_frame, text="Start Logging", command=self.toggle_logging, font=btn_font, bg="#2980b9", fg="white", width=12, state="disabled")
        self.btn_toggle_log.grid(row=0, column=2, padx=3, pady=3)
        self.btn_capture_target = tk.Button(self.controls_frame, text="Capture New", command=self.enter_capture_mode, font=btn_font, bg="#8e44ad", fg="white", width=12, state="disabled")
        self.btn_capture_target.grid(row=0, column=3, padx=3, pady=3)

        tk.Label(self.controls_frame, text="Action:", bg="gold", font=btn_font).grid(row=1, column=0, sticky="e")
        self.required_action_var = tk.StringVar(self.root)
        self.required_action_var.set("Wave Right")
        self.action_dropdown = tk.OptionMenu(self.controls_frame, self.required_action_var, "Wave Right", "Wave Left", "Jump", "Sit", command=self.on_action_change)
        self.action_dropdown.grid(row=1, column=1, sticky="ew")
        self.btn_set_interval = tk.Button(self.controls_frame, text=f"Set Interval ({self.alert_interval}s)", command=self.set_alert_interval, font=btn_font, bg="#7f8c8d", fg="white")
        self.btn_set_interval.grid(row=1, column=2, padx=3, pady=3)
        self.btn_toggle_alert = tk.Button(self.controls_frame, text="Start Alert Mode", command=self.toggle_alert_mode, font=btn_font, bg="#e67e22", fg="white", width=12, state="disabled")
        self.btn_toggle_alert.grid(row=1, column=3, padx=3, pady=3)

        tk.Label(self.listbox_frame, text="Select Targets (Multi-Select):", bg="gold", font=btn_font).pack(anchor="w")
        self.target_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE, height=8, font=('Helvetica', 10))
        self.target_listbox.pack(side="left", fill="both", expand=True)
        self.target_listbox.bind('<<ListboxSelect>>', self.on_listbox_select)
        scrollbar = tk.Scrollbar(self.listbox_frame)
        scrollbar.pack(side="right", fill="y")
        self.target_listbox.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.target_listbox.yview)
        self.btn_apply_targets = tk.Button(self.listbox_frame, text="TRACK SELECTED", command=self.apply_target_selection, font=btn_font, bg="black", fg="gold")
        self.btn_apply_targets.pack(side="bottom", fill="x", pady=2)
        self.btn_refresh = tk.Button(self.listbox_frame, text="Refresh List", command=self.load_targets, font=btn_font, bg="#e67e22", fg="white")
        self.btn_refresh.pack(side="bottom", fill="x", pady=2)

        self.btn_snap = tk.Button(self.controls_frame, text="Snap Photo", command=self.snap_photo, font=btn_font, bg="#d35400", fg="white")
        self.btn_cancel_capture = tk.Button(self.controls_frame, text="Cancel", command=self.exit_capture_mode, font=btn_font, bg="#7f8c8d", fg="white")

        self.load_targets()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def on_close(self):
        self.stop_camera()
        self.root.destroy()

    def load_targets(self):
        logger.info("Loading targets...")
        self.target_map = {}
        target_files = glob.glob("target_*.jpg")
        display_names = []
        for f in target_files:
            try:
                base_name = f.replace(".jpg", "")
                parts = base_name.split('_')
                if len(parts) >= 3:
                    display_name = " ".join(parts[1:-1])
                    self.target_map[display_name] = f
                    display_names.append(display_name)
            except: pass
        
        self.target_listbox.delete(0, tk.END)
        if not display_names:
             self.target_listbox.insert(tk.END, "No targets found")
             self.target_listbox.config(state=tk.DISABLED)
        else:
             self.target_listbox.config(state=tk.NORMAL)
             for name in sorted(list(set(display_names))):
                 self.target_listbox.insert(tk.END, name)

    def on_listbox_select(self, event):
        for widget in self.preview_display.winfo_children(): widget.destroy()
        selections = self.target_listbox.curselection()
        if not selections: return
        
        MAX_PREVIEW = 4
        display_idx = selections[:MAX_PREVIEW]
        cols = 1 if len(display_idx) == 1 else 2
        
        for i, idx in enumerate(display_idx):
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    img = cv2.imread(filename)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (180, 130))
                    pil_img = Image.fromarray(img)
                    imgtk = ImageTk.PhotoImage(image=pil_img)
                    lbl = tk.Label(self.preview_display, image=imgtk, bg="black", text=name, compound="bottom", fg="white", font=("Arial", 9, "bold"))
                    lbl.image = imgtk 
                    lbl.grid(row=i//cols, column=i%cols, padx=5, pady=5)
                except: pass

    def apply_target_selection(self):
        self.targets_status = {} 
        selections = self.target_listbox.curselection()
        if not selections: return
        
        if self.target_listbox.get(selections[0]) == "No targets found": return

        count = 0
        for idx in selections:
            name = self.target_listbox.get(idx)
            filename = self.target_map.get(name)
            if filename:
                try:
                    # Use Memory Scrubber for loading targets too
                    raw_img = cv2.imread(filename)
                    if raw_img is None: continue
                    
                    # Scrub to RGB
                    clean_rgb = scrub_image_memory(raw_img, to_gray=False)
                    if clean_rgb is None: continue

                    encodings = face_recognition.face_encodings(clean_rgb)
                    if encodings:
                        self.targets_status[name] = {
                            "encoding": encodings[0],
                            "tracker": None,
                            "face_box": None, 
                            "visible": False,
                            "last_wave_time": time.time(),
                            "alert_cooldown": 0,
                            "alert_triggered_state": False,
                            "last_logged_action": None,
                            "pose_buffer": deque(maxlen=12),
                            "missing_pose_counter": 0 
                        }
                        count += 1
                except Exception as e:
                    logger.error(f"Error loading {name}: {e}")
        
        if count > 0:
            messagebox.showinfo("Tracking Updated", f"Scanning for {count} targets.")
            if not self.is_alert_mode:
                 self.is_logging = False
                 self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")

    def toggle_alert_mode(self):
        self.is_alert_mode = not self.is_alert_mode
        if self.is_alert_mode:
            self.btn_toggle_alert.config(text="Stop Alert Mode", bg="#c0392b")
            if not self.is_logging: self.toggle_logging()
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time
        else:
            self.btn_toggle_alert.config(text="Start Alert Mode", bg="#e67e22")

    def set_alert_interval(self):
        val = simpledialog.askinteger("Set Interval", "Seconds:", minvalue=1, maxvalue=3600, initialvalue=self.alert_interval)
        if val:
            self.alert_interval = val
            self.btn_set_interval.config(text=f"Set Interval ({self.alert_interval}s)")
            
    def on_action_change(self, value):
        if self.is_alert_mode:
            current_time = time.time()
            for name in self.targets_status:
                self.targets_status[name]["last_wave_time"] = current_time

    def start_camera(self):
        if not self.is_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened(): return
                self.frame_w = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                self.frame_h = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                self.is_running = True
                self.btn_start.config(state="disabled")
                self.btn_stop.config(state="normal")
                self.btn_toggle_log.config(state="normal")
                self.btn_capture_target.config(state="normal")
                self.btn_toggle_alert.config(state="normal")
                self.update_video_feed()
            except: pass

    def stop_camera(self):
        if self.is_running:
            self.is_running = False
            if self.cap: self.cap.release()
            if self.is_logging: self.save_log_to_file()
            self.btn_start.config(state="normal")
            self.btn_stop.config(state="disabled")
            self.video_label.config(image='')

    def toggle_logging(self):
        self.is_logging = not self.is_logging
        if self.is_logging:
            self.temp_log.clear()
            self.btn_toggle_log.config(text="Stop Logging", bg="#c0392b")
        else:
            self.btn_toggle_log.config(text="Start Logging", bg="#2980b9")
            self.save_log_to_file()

    def save_log_to_file(self):
        if self.temp_log:
            try:
                with open(csv_file, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(self.temp_log)
                self.temp_log.clear()
                logger.info("Logs saved.")
            except: pass
            
    def capture_alert_snapshot(self, frame, target_name):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = target_name.replace(" ", "_")
        filename = f"alert_snapshots/alert_{safe_name}_{timestamp}.jpg"
        try:
            cv2.imwrite(filename, frame)
            return filename
        except: return "Error"

    def enter_capture_mode(self):
        if not self.is_running: return
        self.is_in_capture_mode = True
        self.btn_start.grid_remove()
        self.btn_stop.grid_remove()
        self.btn_toggle_log.grid_remove()
        self.btn_capture_target.grid_remove()
        self.btn_snap.grid(row=0, column=0)
        self.btn_cancel_capture.grid(row=0, column=1)

    def exit_capture_mode(self):
        self.is_in_capture_mode = False
        self.btn_snap.grid_remove()
        self.btn_cancel_capture.grid_remove()
        self.btn_start.grid()
        self.btn_stop.grid()
        self.btn_toggle_log.grid()
        self.btn_capture_target.grid()

    # --- SNAP PHOTO (Fixed with Fallbacks) ---
    def snap_photo(self):
        if self.unprocessed_frame is None: return
        
        try:
            # Strategy 1: Scrub Memory to clean RGB (Bytes -> New Array)
            clean_rgb = scrub_image_memory(self.unprocessed_frame, to_gray=False)
            
            face_locations = []
            
            try:
                # Attempt RGB detection
                face_locations = face_recognition.face_locations(clean_rgb)
            except RuntimeError:
                # Strategy 2: Fallback to Grayscale if RGB fails (Dlib is happier with Gray)
                print("DEBUG: RGB Detection failed, switching to Gray Fallback.")
                clean_gray = scrub_image_memory(self.unprocessed_frame, to_gray=True)
                face_locations = face_recognition.face_locations(clean_gray)

            if len(face_locations) == 1:
                self.video_label.config(bg="white")
                self.root.update()
                time.sleep(0.1)
                
                name = simpledialog.askstring("Name", "Enter Name:")
                if name:
                    safe_name = name.strip().replace(" ", "_")
                    cv2.imwrite(f"target_{safe_name}_face.jpg", self.unprocessed_frame)
                    self.load_targets()
                    self.exit_capture_mode()
                    messagebox.showinfo("Success", f"Saved target: {name}")
            elif len(face_locations) == 0:
                messagebox.showwarning("Error", "No face detected.")
            else:
                messagebox.showwarning("Error", "Multiple faces detected.")
                
        except Exception as e:
            logger.error(f"Snapshot error: {e}")
            messagebox.showerror("Error", f"Failed to process: {e}")

    def update_video_feed(self):
        if not self.is_running: return
        ret, frame = self.cap.read()
        if not ret: 
            self.stop_camera()
            return
            
        self.unprocessed_frame = frame.copy()
        
        if self.is_in_capture_mode:
            h, w = frame.shape[:2]
            cv2.ellipse(frame, (w//2, h//2), (100, 130), 0, 0, 360, (0, 255, 255), 2)
        else:
            frame = self.process_tracking(frame)
        
        if self.video_label.winfo_exists():
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            lbl_w = self.video_label.winfo_width()
            lbl_h = self.video_label.winfo_height()
            
            if lbl_w > 10 and lbl_h > 10:
                h, w = frame_rgb.shape[:2]
                scale = min(lbl_w/w, lbl_h/h)
                frame_rgb = cv2.resize(frame_rgb, (int(w*scale), int(h*scale)))
            
            imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
            
        self.root.after(10, self.update_video_feed)

    def process_tracking(self, frame):
        if not self.targets_status:
            cv2.putText(frame, "SELECT TARGETS TO START", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return frame

        self.re_detect_counter += 1
        if self.re_detect_counter > self.RE_DETECT_INTERVAL: self.re_detect_counter = 0
        
        frame_h, frame_w = frame.shape[:2]

        # 1. Update Trackers
        for name, status in self.targets_status.items():
            if status["tracker"]:
                success, box = status["tracker"].update(frame)
                if success:
                    x, y, w, h = [int(v) for v in box]
                    status["face_box"] = (x, y, x+w, y+h)
                    status["visible"] = True
                else:
                    status["visible"] = False
                    status["tracker"] = None

        # 2. Detection
        untracked_targets = [n for n, s in self.targets_status.items() if not s["visible"]]
        if untracked_targets and self.re_detect_counter == 0:
            try:
                # Use Scrubbed RGB for detection
                clean_rgb = scrub_image_memory(frame, to_gray=False)
                
                locs = face_recognition.face_locations(clean_rgb)
                if locs:
                    encs = face_recognition.face_encodings(clean_rgb, locs)
                    matches = []
                    for i, unk_enc in enumerate(encs):
                        for name in untracked_targets:
                            dist = face_recognition.face_distance([self.targets_status[name]["encoding"]], unk_enc)[0]
                            if dist < 0.55: matches.append((dist, i, name))
                    
                    matches.sort(key=lambda x: x[0])
                    assigned = set()
                    
                    for dist, idx, name in matches:
                        if idx in assigned or self.targets_status[name]["visible"]: continue
                        assigned.add(idx)
                        t, r, b, l = locs[idx]
                        
                        tracker = create_tracker()
                        if tracker:
                            tracker.init(frame, (l, t, r-l, b-t))
                            self.targets_status[name]["tracker"] = tracker
                            self.targets_status[name]["face_box"] = (l, t, r, b)
                            self.targets_status[name]["visible"] = True
                            self.targets_status[name]["missing_pose_counter"] = 0
            except Exception: pass

        # 3. Draw & Pose
        for name, status in self.targets_status.items():
            if status["visible"]:
                fx1, fy1, fx2, fy2 = status["face_box"]
                face_w = fx2 - fx1
                face_cx = fx1 + (face_w // 2)
                
                # Body Box
                bx1 = max(0, int(face_cx - (face_w * 3)))
                bx2 = min(frame_w, int(face_cx + (face_w * 3)))
                by1 = max(0, int(fy1 - (face_w * 0.5)))
                by2 = frame_h 

                pose_found = False
                if bx1 < bx2 and by1 < by2:
                    crop = frame[by1:by2, bx1:bx2]
                    if crop.size != 0:
                        rgb_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                        rgb_crop.flags.writeable = False
                        res = self.holistic_crop.process(rgb_crop)
                        rgb_crop.flags.writeable = True
                        
                        if res.pose_landmarks:
                            pose_found = True
                            status["missing_pose_counter"] = 0
                            draw_styled_landmarks(crop, res)
                            act = classify_action(res.pose_landmarks.landmark, by2-by1, bx2-bx1)
                            status["pose_buffer"].append(act)
                            
                            curr_act = act
                            if len(status["pose_buffer"]) >= 8:
                                curr_act = Counter(status["pose_buffer"]).most_common(1)[0][0]
                            
                            req_act = self.required_action_var.get()
                            if curr_act == req_act:
                                if self.is_alert_mode: status["last_wave_time"] = time.time()
                                if self.is_logging and status["last_logged_action"] != req_act:
                                    self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, curr_act, "SAFE", "N/A"))
                                    status["last_logged_action"] = req_act
                            
                            cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
                            cv2.putText(frame, f"{name}: {curr_act}", (bx1, by1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                if not pose_found:
                    status["missing_pose_counter"] += 1
                    if status["missing_pose_counter"] > 10:
                        status["tracker"] = None
                        status["visible"] = False
            
            # Alert Overlay
            if self.is_alert_mode:
                diff = time.time() - status["last_wave_time"]
                rem = max(0, self.alert_interval - diff)
                col = (0,255,0) if rem > 3 else (0,0,255)
                y_off = 50 + (list(self.targets_status.keys()).index(name)*30)
                txt = "OK" if status["visible"] else "MISSING"
                cv2.putText(frame, f"{name} ({txt}): {rem:.1f}s", (frame_w-300, y_off), cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)
                
                if diff > self.alert_interval and (time.time() - status["alert_cooldown"] > 2.5):
                    play_siren_sound()
                    status["alert_cooldown"] = time.time()
                    self.capture_alert_snapshot(frame, name)
                    if self.is_logging:
                        self.temp_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), name, "MISSING/IDLE", "ALERT", "CAPTURED"))

        return frame

if __name__ == "__main__":
    app = PoseApp()

d:\venv3114\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
2025-11-19 16:10:23,856 - INFO - Loading targets...
2025-11-19 16:10:44,404 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: RGB Detection failed, switching to Gray Fallback.


2025-11-19 16:10:47,246 - ERROR - Snapshot error: Unsupported image type, must be 8bit gray or RGB image.


DEBUG: RGB Detection failed, switching to Gray Fallback.
